In [ ]:
import os
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib as plt

def rodar_simulador(taxa_chegada = 40, tempo_medio_servico = 0.1, num_servidores = 1, tempo_observacao = 1):
    comando = [
        "java", "-cp", "bin:lib/*", "ServidorWeb",
        str(taxa_chegada), str(tempo_medio_servico),
        str(num_servidores), str(tempo_medio_servico),
        str(tempo_observacao)]
    resultado = subprocess.run(comando)
    return resultado.stdout


In [12]:
saida = rodar_simulador()
print(saida)

TaxaDeChegadaMedia TempoDeServicoMedio NumServidores RequisicoesSubmetidas RequisicoesConcluidas TempoMedioDeResposta TamanhoMedioDaFila
40,000000 0,100000 1 0 0 NaN 0,000000
None
